# Import Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras 
from keras.layers import GlobalAveragePooling2D, Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.applications import resnet50
from keras.regularizers import l2
from keras.initializers import GlorotNormal
from keras.callbacks import ModelCheckpoint
from keras.metrics import Precision, Recall, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, auc, roc_auc_score, roc_curve 
from sklearn.preprocessing import label_binarize
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import itertools
from tabulate import tabulate
from numpy import interp
from pathlib import Path
from PIL import Image
from datetime import datetime
import pytz
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

# Set Timezone

In [ ]:
MYT = pytz.timezone('Asia/Kuala_Lumpur')

In [ ]:
start_time = datetime.now(MYT)
print("Start Time:", start_time.strftime('%Y/%m/%d %H:%M:%S'))

# Set GPU

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPU available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.compat.v1.Session(config=config)

# Set Hyperparameters

In [ ]:
model_regularizer = 1000
model_lr = 1e-6
model_epochs = 50

deeptune_model_regularizer = 100
deeptune_model_lr = 1e-7
deeptune_model_epochs = 150

fold = 1
run = f'Fold{fold}'

# Defining Dataset

In [ ]:
def read_pil_image(img_path, height, width):
        with open(img_path, 'rb') as f:
            return np.array(Image.open(f).convert('RGB').resize((width, height)))

def load_all_images(dataset_path, height, width):
    return np.array([read_pil_image(str(p), height, width) for p in Path(dataset_path).rglob("*.*")])

### For Cross Validation / Actual Classification Task

In [ ]:
num_classes = 4
height = width = 224

In [ ]:
train_path = f'../Dataset/Cross_Validation/Fold{fold}/train'
val_path = f'../Dataset/Cross_Validation/Fold{fold}/val'
test_path = f'../Dataset/Cross_Validation/Fold{fold}/test'

#### Run these without zero-meaning data:

train_batch = ImageDataGenerator(preprocessing_function=resnet50.preprocess_input).flow_from_directory(
    directory = train_path,
    target_size = (224,224),
    batch_size = 16
)

val_batch = ImageDataGenerator(preprocessing_function=resnet50.preprocess_input).flow_from_directory(
    directory = val_path,
    target_size = (224,224),
    batch_size = 16
)

test_batch = ImageDataGenerator(preprocessing_function=resnet50.preprocess_input).flow_from_directory(
    directory = test_path,
    target_size = (224,224),
    batch_size = 16,
    shuffle = False
)

#### Run these to zero-mean data:

In [ ]:
data_prep_start_time = datetime.now(MYT)
print("Start Time (Hold-out Validation Data Preparation):", data_prep_start_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)
train_datagen.fit(load_all_images(train_path, height, width))

In [ ]:
train_batch = train_datagen.flow_from_directory(
    directory = train_path,
    target_size = (height, width),
    batch_size = 16
)

val_batch = train_datagen.flow_from_directory(
    directory = val_path,
    target_size = (224,224),
    batch_size = 16
)

test_batch = train_datagen.flow_from_directory(
    directory = test_path,
    target_size = (224,224),
    batch_size = 16,
    shuffle = False
)

In [ ]:
data_prep_end_time = datetime.now(MYT)
print("End Time (Hold-out Validation Data Preparation):", data_prep_end_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
print("Total Run Time (Hold-out Validation Data Preparation):", data_prep_end_time-data_prep_start_time)

### For Deep Tuning Task

In [ ]:
deeptuning_num_classes = 2
height = width = 224

In [ ]:
deeptuning_train_path = '../Dataset/Deep_Tuning/train'
deeptuning_val_path = '../Dataset/Deep_Tuning/val'

#### Run these without zero-meaning data:

deeptuning_train_batch = ImageDataGenerator(preprocessing_function=resnet50.preprocess_input).flow_from_directory(
    directory = deeptuning_train_path,
    target_size = (224,224),
    batch_size = 5
)

deeptuning_val_batch = ImageDataGenerator(preprocessing_function=resnet50.preprocess_input).flow_from_directory(
    directory = deeptuning_val_path,
    target_size = (224,224),
    batch_size = 5
)

#### Run these to zero-mean data:

In [ ]:
deeptuning_data_prep_start_time = datetime.now(MYT)
print("Start Time (Deep Tuning Data Preparation):", deeptuning_data_prep_start_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
deeptuning_train_datagen = ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)
deeptuning_train_datagen.fit(load_all_images(deeptuning_train_path, height, width))

In [ ]:
deeptuning_train_batch = deeptuning_train_datagen.flow_from_directory(
    directory = deeptuning_train_path,
    target_size = (224,224),
    batch_size = 64
)

deeptuning_val_batch = deeptuning_train_datagen.flow_from_directory(
    directory = deeptuning_val_path,
    target_size = (224,224),
    batch_size = 64
)

In [ ]:
deeptuning_data_prep_end_time = datetime.now(MYT)
print("End Time (Deep Tuning Data Preparation):", deeptuning_data_prep_end_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
print("Total Run Time (Deep Tuning Data Preparation):", deeptuning_data_prep_end_time-deeptuning_data_prep_start_time)

#### Calculate class weights due to imbalanced dataset:

class_weights = class_weight.compute_class_weight(
    'balanced',
    np.unique(deeptuning_train_batch.classes),
    deeptuning_train_batch.classes
)
class_weights = {i : class_weights[i] for i in range(deeptuning_num_classes)}
class_weights

# Model 1

### Building Model 1

In [ ]:
base_model1 = resnet50.ResNet50(include_top=False, weights='imagenet')

In [ ]:
base_model1.summary()

In [ ]:
x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)
prediction_layer1 = Dense(
    num_classes, 
    activation='softmax', 
    kernel_regularizer=l2(model_regularizer), 
    kernel_initializer=GlorotNormal())(x1)
model1 = Model(inputs=base_model1.input, outputs=prediction_layer1)

In [ ]:
model1.summary()

In [ ]:
for layer in base_model1.layers:
    layer.trainable = True

In [ ]:
model1.summary()

In [ ]:
model1.compile(
    optimizer = Adam(lr=model_lr),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

### Training Model 1

In [ ]:
model1_training_start_time = datetime.now(MYT)
print("Start Time (Model 1 Training):", model1_training_start_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
checkpoint1 = tf.keras.callbacks.ModelCheckpoint(
    f'model1_weights_zeromean_{run}.h5', 
    monitor='val_accuracy', 
    verbose=1,
    save_best_only=True, 
    save_weights_only=True,
    mode='max', 
    period=1
)

In [ ]:
history1 = model1.fit(
    x = train_batch,
    validation_data = val_batch,
    epochs = model_epochs,
    verbose = 2,
    callbacks = [checkpoint1]
)
history1

In [ ]:
model1_training_end_time = datetime.now(MYT)
print("End Time (Model 1 Training):", model1_training_end_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
print("Total Run Time (Model 1 Training):", model1_training_end_time-model1_training_start_time)

In [ ]:
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

### Testing Model 1

In [ ]:
model1.load_weights(f'model1_weights_zeromean_{run}.h5')

In [ ]:
predictions1 = model1.predict(x=test_batch, verbose=0)

# Model 2

### Deep Tuning Model 2

In [ ]:
deeptuning_base_model = resnet50.ResNet50(include_top=False, weights='imagenet')

In [ ]:
deeptuning_base_model.summary()

In [ ]:
deeptuning_base_model.get_weights()

In [ ]:
deeptuning_x = deeptuning_base_model.output
deeptuning_x = GlobalAveragePooling2D()(deeptuning_x)
deeptuning_predictions = Dense(
    deeptuning_num_classes, 
    activation='softmax', 
    kernel_regularizer=l2(deeptune_model_regularizer), 
    kernel_initializer=GlorotNormal())(deeptuning_x)
deeptune_model = Model(inputs=deeptuning_base_model.input, outputs=deeptuning_predictions)

In [ ]:
deeptune_model.summary()

In [ ]:
for layer in deeptuning_base_model.layers:
    layer.trainable = True

In [ ]:
deeptune_model.summary()

In [ ]:
deeptune_model.compile(
    optimizer = Adam(lr=deeptune_model_lr),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
deeptuning_training_start_time = datetime.now(MYT)
print("Start Time (Deep Tuning Training):", deeptuning_training_start_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
deeptune_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'deeptuned_model.h5', 
    monitor='val_accuracy', 
    verbose=1,
    save_best_only=True, 
    mode='max', 
    period=1
)

In [ ]:
deeptuning_history = deeptune_model.fit(
    x = deeptuning_train_batch,
    validation_data=deeptuning_val_batch,
    epochs = deeptune_model_epochs,
    verbose = 2,
    callbacks = [deeptune_checkpoint]
    #class_weight = class_weights
)
deeptuning_history

In [ ]:
deeptuning_training_end_time = datetime.now(MYT)
print("End Time (Deep Tuning Training):", deeptuning_training_end_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
print("Total Run Time (Deep Tuning Training):", deeptuning_training_end_time-deeptuning_training_start_time)

In [ ]:
plt.plot(deeptuning_history.history['accuracy'])
plt.plot(deeptuning_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(deeptuning_history.history['loss'])
plt.plot(deeptuning_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
deeptune_model.get_weights()

### Bulding Model 2

In [ ]:
base_model2 = load_model('deeptuned_model.h5')

In [ ]:
base_model2.summary()

In [ ]:
temp_model = Model(inputs=base_model2.input, outputs=base_model2.layers[-3].output)
temp_model.summary()

In [ ]:
x2 = temp_model.output
x2 = GlobalAveragePooling2D()(x2)
prediction_layer2 = Dense(
    num_classes, 
    activation='softmax', 
    kernel_regularizer=l2(model_regularizer), 
    kernel_initializer=GlorotNormal())(x2)
model2 = Model(inputs=temp_model.input, outputs=prediction_layer2)

In [ ]:
model2.summary()

In [ ]:
model2.get_weights()

In [ ]:
for layer in base_model2.layers:
    layer.trainable = True

In [ ]:
model2.summary()

In [ ]:
model2.compile(
    optimizer = Adam(lr=model_lr),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

### Training Model 2

In [ ]:
model2_training_start_time = datetime.now(MYT)
print("Start Time (Model 2 Training):", model2_training_start_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
checkpoint2 = tf.keras.callbacks.ModelCheckpoint(
    f'model2_weights_zeromean_{run}.h5', 
    monitor='val_accuracy', 
    verbose=1,
    save_best_only=True, 
    save_weights_only=True,
    mode='max', 
    period=1
)

In [ ]:
history2 = model2.fit(
    x = train_batch,
    validation_data = val_batch,
    epochs = model_epochs,
    verbose = 2,
    callbacks = [checkpoint2]
)
history2

In [ ]:
model2_training_end_time = datetime.now(MYT)
print("End Time (Model 2 Training):", model2_training_end_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
print("Total Run Time (Model 2 Training):", model2_training_end_time-model2_training_start_time)

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

### Testing Model 2

In [ ]:
model2.load_weights(f'model2_weights_zeromean_{run}.h5')

In [ ]:
predictions2 = model2.predict(x=test_batch, verbose=0)

# Metrics Evaluation for Model 1 and Model 2

### Defining Functions

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def specificity(y_true, y_pred):
    m = TrueNegatives()
    m.update_state(y_true, y_pred)
    final_tn = m.result().numpy()
    
    n = FalsePositives()
    n.update_state(y_true, y_pred)
    final_fp = n.result().numpy()
    
    return final_tn / (final_tn + final_fp)

### Confusion Matrix

In [ ]:
test_batch.class_indices

In [ ]:
test_labels = test_batch.classes
cm_plot_labels = ['Bacterial Pneumonia','COVID','Normal','Viral Pneumonia']

#### Model 1 Confusion Matrix Construction:

In [ ]:
cm1 = confusion_matrix(y_true=test_labels, y_pred=predictions1.argmax(axis=-1))

#### Model 2 Confusion Matrix Construction:

In [ ]:
cm2 = confusion_matrix(y_true=test_labels, y_pred=predictions2.argmax(axis=-1))

#### Plot Confusion Matrix:

In [ ]:
plot_confusion_matrix(cm=cm1, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
plot_confusion_matrix(cm=cm2, classes=cm_plot_labels, title='Confusion Matrix')

### Accuracy, Precision, Recall, F1-Score, Specificity, ROC-AUC Score

In [ ]:
y_true=test_labels
y_true_binarized=label_binarize(y_true, classes=[0,1,2,3])

#### Model 1 Metrics Computation:

In [ ]:
y_pred1=predictions1.argmax(axis=-1)
y_pred_binarized1=label_binarize(y_pred1, classes=[0,1,2,3])

In [ ]:
accuracy1 = accuracy_score(y_true, y_pred1)
precision1 = precision_score(y_true, y_pred1, average=None)
recall1 = recall_score(y_true, y_pred1, average=None)
f1_metrics_score1 = f1_score(y_true, y_pred1, average=None)
macro_roc_auc_score1 = roc_auc_score(y_true, predictions1, multi_class="ovr")

In [ ]:
specificity_score1 = []
roc_auc_metrics_score1 = []

for i in range(num_classes):
    specificity_score1.append(specificity(y_true_binarized[:,i], y_pred_binarized1[:,i]))
    roc_auc_metrics_score1.append(roc_auc_score(y_true_binarized[:,i], predictions1[:,i]))

#### Model 2 Metrics Computation:

In [ ]:
y_pred2=predictions2.argmax(axis=-1)
y_pred_binarized2=label_binarize(y_pred2, classes=[0,1,2,3])

In [ ]:
accuracy2 = accuracy_score(y_true, y_pred2)
precision2 = precision_score(y_true, y_pred2, average=None)
recall2 = recall_score(y_true, y_pred2, average=None)
f1_metrics_score2 = f1_score(y_true, y_pred2, average=None)
macro_roc_auc_score2 = roc_auc_score(y_true, predictions2, multi_class="ovr")

In [ ]:
specificity_score2 = []
roc_auc_metrics_score2 = []

for i in range(num_classes):
    specificity_score2.append(specificity(y_true_binarized[:,i], y_pred_binarized2[:,i]))
    roc_auc_metrics_score2.append(roc_auc_score(y_true_binarized[:,i], predictions2[:,i]))

#### Display Metrics:

In [ ]:
header = ['Class', 'Precision', 'Recall', 'F1-Score', 'Specificity', 'ROC-AUC Score']
decimal_formatting = '.5f'

In [ ]:
data1 = [[],[],[],[]]

for i in range(num_classes):
    data1[i].append(cm_plot_labels[i])
    data1[i].append(format(precision1[i], decimal_formatting))
    data1[i].append(format(recall1[i], decimal_formatting))
    data1[i].append(format(f1_metrics_score1[i], decimal_formatting))
    data1[i].append(format(specificity_score1[i], decimal_formatting))
    data1[i].append(format(roc_auc_metrics_score1[i], decimal_formatting))

print('Model 1 Metrics Summary:')
print(f'Accuracy: {format(accuracy1, decimal_formatting)}\n')
print(f'{tabulate(data1, headers=header)}\n')
print(f'Macro-averaged ROC-AUC Score: {format(macro_roc_auc_score1, decimal_formatting)}')

In [ ]:
data2 = [[],[],[],[]]

for i in range(num_classes):
    data2[i].append(cm_plot_labels[i])
    data2[i].append(format(precision2[i], decimal_formatting))
    data2[i].append(format(recall2[i], decimal_formatting))
    data2[i].append(format(f1_metrics_score2[i], decimal_formatting))
    data2[i].append(format(specificity_score2[i], decimal_formatting))
    data2[i].append(format(roc_auc_metrics_score2[i], decimal_formatting))

print('Model 2 Metrics Summary:')
print(f'Accuracy: {format(accuracy2, decimal_formatting)}\n')
print(f'{tabulate(data2, headers=header)}\n')
print(f'Macro-averaged ROC-AUC Score: {format(macro_roc_auc_score2, decimal_formatting)}')

### Plot ROC-AUC Curve for Comparison between Model 1 and Model 2

#### Model 1 ROC-AUC Curve Computation:

In [ ]:
fpr1 = dict()
tpr1 = dict()
roc_auc1 = dict()

for i in range(num_classes):
    fpr1[i], tpr1[i], _ = roc_curve(y_true_binarized[:, i], predictions1[:, i])
    roc_auc1[i] = auc(fpr1[i], tpr1[i])

# Compute macro-average ROC curve and ROC area
all_fpr1 = np.unique(np.concatenate([fpr1[i] for i in range(num_classes)]))
mean_tpr1 = np.zeros_like(all_fpr1)
for i in range(num_classes):
    mean_tpr1 += interp(all_fpr1, fpr1[i], tpr1[i])
mean_tpr1 /= num_classes
fpr1["macro"] = all_fpr1
tpr1["macro"] = mean_tpr1
roc_auc1["macro"] = auc(fpr1["macro"], tpr1["macro"])

# Compute micro-average ROC curve and ROC area
fpr1["micro"], tpr1["micro"], _ = roc_curve(y_true_binarized.ravel(), predictions1.ravel())
roc_auc1["micro"] = auc(fpr1["micro"], tpr1["micro"])

#### Model 2 ROC-AUC Curve Computation:

In [ ]:
fpr2 = dict()
tpr2 = dict()
roc_auc2 = dict()

for i in range(num_classes):
    fpr2[i], tpr2[i], _ = roc_curve(y_true_binarized[:, i], predictions2[:, i])
    roc_auc2[i] = auc(fpr2[i], tpr2[i])

# Compute macro-average ROC curve and ROC area
all_fpr2 = np.unique(np.concatenate([fpr2[i] for i in range(num_classes)]))
mean_tpr2 = np.zeros_like(all_fpr2)
for i in range(num_classes):
    mean_tpr2 += interp(all_fpr2, fpr2[i], tpr2[i])
mean_tpr2 /= num_classes
fpr2["macro"] = all_fpr2
tpr2["macro"] = mean_tpr2
roc_auc2["macro"] = auc(fpr2["macro"], tpr2["macro"])

# Compute micro-average ROC curve and ROC area
fpr2["micro"], tpr2["micro"], _ = roc_curve(y_true_binarized.ravel(), predictions2.ravel())
roc_auc2["micro"] = auc(fpr2["micro"], tpr2["micro"])

#### Plot ROC-AUC Curves:

In [ ]:
wspace=0.3
hspace=0.5
subtitle_fontsize = 18
subtitle_fontweight = 'semibold'
maintitle_fontsize = 32
maintitle_fontweight = 'bold'

plt.figure(figsize=(15,10))

for i in range(num_classes):
    plt.subplot(2, 3, i+1)
    plt.subplots_adjust(wspace=wspace, hspace=hspace)
    plt.plot(
        fpr1[i],
        tpr1[i],
        label=f'Model 1 (AUC = {roc_auc1[i]:.3f})',
        color='navy',
        linewidth=2
    )
    plt.plot(
        fpr2[i],
        tpr2[i],
        label=f'Model 2 (AUC = {roc_auc2[i]:.3f})',
        color='darkorange',
        linewidth=2
    )
    plt.title(cm_plot_labels[i], fontsize=subtitle_fontsize, fontweight=subtitle_fontweight)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    
plt.subplot(2, 3, 5)
plt.subplots_adjust(wspace=wspace, hspace=hspace)
plt.plot(
    fpr1["micro"],
    tpr1["micro"],
    label=f'Model 1 (AUC = {roc_auc1["micro"]:.3f})',
    color='navy',
    linestyle=':',
    linewidth=4
)
plt.plot(
    fpr2["micro"],
    tpr2["micro"],
    label=f'Model 2 (AUC = {roc_auc2["micro"]:.3f})',
    color='darkorange',
    linestyle=':',
    linewidth=4
)
plt.title('Micro-averaged', fontsize=subtitle_fontsize, fontweight=subtitle_fontweight)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')

plt.subplot(2, 3, 6)
plt.subplots_adjust(wspace=wspace, hspace=hspace)
plt.plot(
    fpr1["macro"],
    tpr1["macro"],
    label=f'Model 1 (AUC = {roc_auc1["macro"]:.3f})',
    color='navy',
    linestyle=':',
    linewidth=4
)
plt.plot(
    fpr2["macro"],
    tpr2["macro"],
    label=f'Model 2 (AUC = {roc_auc2["macro"]:.3f})',
    color='darkorange',
    linestyle=':',
    linewidth=4
)
plt.title('Macro-averaged', fontsize=subtitle_fontsize, fontweight=subtitle_fontweight)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')

plt.suptitle('ROC Curve', fontsize=maintitle_fontsize, fontweight=maintitle_fontweight)
plt.show()

In [ ]:
end_time = datetime.now(MYT)
print("End Time:", end_time.strftime('%Y/%m/%d %H:%M:%S'))

In [ ]:
print("Total Run Time:", end_time-start_time)